In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import os

# Verify file existence (optional)
if os.path.exists("movie_review.csv"):
    print("File found!")
    data = pd.read_csv("movie_review.csv")
else:
    print("Error: File not found!")
# Cargar el conjunto de datos


# Preprocesamiento de datos
# Supongamos que tienes una columna 'texto' con las reseñas y una columna 'etiqueta' con las etiquetas (0 para negativo, 1 para positivo).
# Separar los datos en características (X) y etiquetas (y)
X = data['text']
y = data['tag']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

# Paso 4: Vectorización de texto
vectorizer = TfidfVectorizer(max_features=5000) # Convertir el texto en características numéricas (TF-IDF)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Paso 5: Entrenar un modelo de clasificación
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Paso 6: Realizar predicciones en el conjunto de prueba
y_pred = clf.predict(X_test)

# Paso 7: Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Precisión del modelo: {accuracy}")
print("Informe de clasificación:")
print(report)

File found!
Precisión del modelo: 0.6697311495673671
Informe de clasificación:
              precision    recall  f1-score   support

         neg       0.67      0.65      0.66      6371
         pos       0.67      0.69      0.68      6573

    accuracy                           0.67     12944
   macro avg       0.67      0.67      0.67     12944
weighted avg       0.67      0.67      0.67     12944



# Mejoras al algoritmo
1. Preprocesamiento de texto en las reseñas originales.
2. Modificación de parámetros en la vectorización TF-IDF

In [ ]:
# 1. Pre-procesamiento del texto
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenizar el texto
    tokens = word_tokenize(text)
    # Eliminar stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lematizar palabras
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Realizar stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    # Unir tokens nuevamente en texto
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [ ]:
# Aplicar preprocesamiento de texto a todas las reseñas
for i in range(data['text'].shape[0]):
  data.loc[i, 'preprocessed_text'] = preprocess_text(data.loc[i, 'text'])

In [ ]:
# Separar los datos en características (X) y etiquetas (y)
X = data['preprocessed_text']
y = data['tag']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 4: Vectorización de texto
vectorizer = TfidfVectorizer(max_features=5000) # Convertir el texto en características numéricas (TF-IDF)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Paso 5: Entrenar un modelo de clasificación
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Paso 6: Realizar predicciones en el conjunto de prueba
y_pred = clf.predict(X_test)

# Paso 7: Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Precisión del modelo: {accuracy}")
print("Informe de clasificación:")
print(report)

In [ ]:
# 2. Modificación de parámetros en Tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

algorithms = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# Definir los parámetros a ajustar
parameters = {
    'tfidf__max_features': [1000, 5000, 10000],  # Tamaño máximo del vocabulario
    'tfidf__min_df': [1, 2, 5],                  # Frecuencia mínima de documento
    # Otros parámetros del TfidfVectorizer...
}

# Realizar la búsqueda en cuadrícula para encontrar los mejores parámetros
grid_search = GridSearchCV(algorithms, parameters, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros encontrados
best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)

# Evaluar el rendimiento del modelo con los mejores parámetros en el conjunto de prueba
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Precisión del modelo con mejores parámetros: {accuracy}")
print("Informe de clasificación:")
print(report)


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

results = pd.DataFrame(grid_search.cv_results_)

param_names = list(parameters.keys())
param_values = [parameters[param] for param in param_names]
scores = np.array(results.mean_test_score).reshape(len(param_values[0]), len(param_values[1]))

sns.set()
plt.figure(figsize=(8, 4))
c = sns.color_palette("mako", as_cmap=True)
c = c.reversed()
sns.heatmap(scores, annot=True, cmap=c, xticklabels=param_values[1], yticklabels=param_values[0])
plt.xlabel(param_names[1])
plt.ylabel(param_names[0])
plt.title("Combinación de parámetros")
plt.show()

In [ ]:
# Separar los datos en características (X) y etiquetas (y)
X = data['preprocessed_text']
y = data['tag']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

best_tfidf_params = {
    'max_features': 10000,  # Ejemplo de los mejores parámetros obtenidos para TF-IDF
    'min_df': 1            # Ejemplo de los mejores parámetros obtenidos para TF-IDF
}

vectorizer = TfidfVectorizer(**best_tfidf_params)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Paso 5: Entrenar un modelo de clasificación
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Paso 6: Realizar predicciones en el conjunto de prueba
y_pred = clf.predict(X_test)

# Paso 7: Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Precisión del modelo: {accuracy}")
print("Informe de clasificación:")
print(report)